# This notebook takes the data and conducts the Automation Shock

In [34]:
# Required packages
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import datetime as dt
import community
from shapely.geometry import Polygon


import importlib
import ddom

import random
import math

import cmocean as cmo

In [35]:
# Read the data from data_processing.ipynb
sa_calibration_data = pd.read_csv('../Data_Labour/calibration_data.csv')
employment_SSYK = pd.read_csv('../Data_Labour/occupational_employment.csv', sep = ',')
SSYK_shock = pd.read_csv('../Data_Labour/occupation_shock.csv', sep = ',', index_col = 0)
hours_data = pd.read_csv('../Data_Labour/hours_data.csv', sep = ',', index_col = 0)

In [33]:
importlib.reload(ddom)
employment = employment_SSYK[['ssyk3', '2018']]
employment = {str(employment['ssyk3'].iloc[i]):employment['2018'].iloc[i] for i in range(len(employment))}
node_names = G.nodes()

# setup network
employed = {str(name):e for name,e in employment.items() if str(name) in node_names}
unemployed = {name:0 for name in node_names}
vacancies = {name:[] for name in node_names}

target_demand = {str(name):e for name,e in employment.items() if str(name) in node_names}
of_data = SSYK_shock.to_dict()['Computerisation Probability']
of_data = {str(code):prob for code, prob in of_data.items()}

attributes = {'employed':employed, 'unemployed':unemployed, 'vacancies':vacancies,
                'target_demand':target_demand, 'comp_prob':of_data}

# 0.026 delta_u: 0.011 delta_nu: 0.00042 gamma_u: 0.174 timestep: 16.6

# Parameters
a = 0.026
delta_u = 0.011
delta_nu = 0.00042
gamma_u = 0.174
gamma_nu = gamma_u
timestep = 16.6
period = 10.25
shock_period = 30
k = 0.79
avg_hours_0 = hours_data.loc[2018,'average_hours/year']
t_0 = 15


output = ddom.deterministic_simulation(G, delta_u, delta_nu, gamma_u, gamma_nu, timestep, period, shock_period, k, avg_hours_0, t_0, attributes, True, calibration = True)

ERROR: Should not calculate long term unemployment during a calibration run


In [47]:
print(np.arange(0.005, 0.012, 0.002))

[0.005 0.007 0.009 0.011]


In [46]:
employment = employment_SSYK[['ssyk3', '2016']]
employment = {str(employment['ssyk3'].iloc[i]):employment['2016'].iloc[i] for i in range(len(employment))}
node_names = G.nodes()

importlib.reload(ddom)


# setup network
employed = {str(name):e for name,e in employment.items() if str(name) in node_names}
unemployed = {name:0 for name in node_names}
vacancies = {name:[] for name in node_names}

target_demand = {str(name):e for name,e in employment.items() if str(name) in node_names}
of_data = SSYK_shock.to_dict()['Computerisation Probability']
of_data = {str(code):prob for code, prob in of_data.items()}

attributes = {'employed':employed, 'unemployed':unemployed, 'vacancies':vacancies,
                'target_demand':target_demand, 'comp_prob':of_data}

# Parameters short calibration
#delta_u = 0.009
#delta_nu = 0.002
#gamma_u = 0.1
#gamma_nu = gamma_u
#timestep = 14

# Paramters long calibration (currently running)
delta_u = 0.011
delta_nu = 0.005
gamma_u = 0.1
gamma_nu = gamma_u
timestep = 8

# General parameters
period = 10.25
shock_period = 30
k = 0.79
avg_hours_0 = hours_data.loc[2016,'average_hours/year']
t_0 = shock_period/2

vac_datalist = []
u_datalist = []
e_datalist = []
td_datalist = []
lt_u_datalist = []

for i in range(1,5):
    G = nx.read_graphml('../Data_Labour/Occ_mob_sweden.graphml')
    output_abm = ddom.simulation(G, delta_u, delta_nu, gamma_u, gamma_nu, timestep, 
                                 period, shock_period, k, avg_hours_0, t_0, attributes, complete_network = True)
    vac_data = pd.DataFrame(output_abm['vacancy_data'])
    u_data = pd.DataFrame(output_abm['unemployment_data'])
    e_data = pd.DataFrame(output_abm['employment_data'])
    td_data = pd.DataFrame(output_abm['target_demand_data'])
    vac_datalist.append(vac_data)
    u_datalist.append(u_data)
    e_datalist.append(e_data)
    td_datalist.append(td_data)
    td_data.to_csv('../Data_Labour/simulation_output/c_abm_simulation_short_'+str(i)+'.csv', sep = ',', index = False)
    vac_data.to_csv('../Data_Labour/simulation_output/c_abm_vac_simulation_short_'+str(i)+'.csv', sep = ',', index = False)
    e_data.to_csv('../Data_Labour/simulation_output/c_abm_emp_simulation_'+str(i)+'.csv', sep = ',', index = False)
    u_data.to_csv('../Data_Labour/simulation_output/c_abm_unemp_simulation_'+str(i)+'.csv', sep = ',', index = False)

Simulation started at:  2020-05-17 15:21:51.986129
Simulation completed 2020-05-17 20:03:49.491633
Simulation took:  4:41:57.512389
Simulation started at:  2020-05-17 20:03:50.795827


KeyboardInterrupt: 